# Neural Network - Cancer Data

This project mainly focuses on the understanding and implementation of Neural Networks in Python from scratch. I have used Wisconsin Breast Cancer Dataset for this project. The main goal of this project is to predict the cancer malignancy with a neural network.

## Dataset Information:

   - Number of Attributes - 11
        - id
        - clump_thickness
        - cell_uniformity
        - cellshape_uniformity
        - adhesion
        - cell_size
        - nuclei
        - chromatin
        - nucleoli
        - mitoses
        - Class

We will be predicting the Class attribute based on the other attributes mentioned above.









## Importing the required libraries

In [1]:
import pandas as pd
import numpy as np

## Implementation of Neural Network:

### Structure of a Neural Network:
   - Neural Network consists of an input layer, hidden layers and an output layer.
   - Weights and biases associated with each layer.
   - Activation function associated with each hidden layer.


## Weight Initialization:

In this function the weights are intialized randomly in the range of [-1, 1]. This function takes nodes as an input which is nothing but the number of nodes in each layer and it returns a multi dimensional array called weights and each element in the array represents a connection between previous and current layer. Biases are also included along with the weights.

In [2]:
def weight_initializer(nodes):
    hidden_layers, weights = len(nodes), []
    
    for i in range(1, hidden_layers):
        weight = [[np.random.uniform(-1, 1) for k in range(nodes[i-1] + 1)] for j in range(nodes[i])]
        weights.append(np.matrix(weight))
    
    return weights

## Neural Network:

This function is used to train the network for the given number of iterations. It takes training data, validation data, number of iterations, learning rate and list of integers as inputs. Accuracies obtained from the train and validation data are printed for each iteration.

In [3]:
def neural_network(X_train, Y_train, X_val=None, Y_val=None, iterations=10, nodes=[], rate=0.15):
    hiddenLayers = len(nodes) - 1
    weights = weight_initializer(nodes)

    for iteration in range(1, iterations+1):
        weights = train_neuralNet(X_train, Y_train, rate, weights)

        #Print the accuracy of training and validation after every 20 iterations
        if(iteration % 20 == 0):
            print("Iteration {}".format(iteration))
            print("Accuracy on train data:{}".format(model_accuracy(X_train, Y_train, weights)))
            if X_val.any():
                print("Accuracy on validation data:{}".format(model_accuracy(X_val, Y_val, weights)))
                        
    return weights

##Note :

The weights need to be continuously adjusted across each iteration to increase the accuracy of the network. In each iteration the network is trained using forward/backward propagation algorithm. First, the input is passed through the network and output is calculated and then, according to the error of output, the weights are updated backwards. So basically, the error is propagated backward.

## Feed Forward Propagation:

Each layer receives an input and computes the output which is calculated by passing the dot product of input and weights of the layer to a activation function. I used "Sigmoid" activation function. The output of each layer is the input for the next layer.

In this network, the information moves in a single direction input to hidden nodes followed by output nodes. No cycles or loops detected in the network. In this function, weights, number of hidden layers are taken as inputs.







In [4]:
def Feed_Forward(x, weights, hidden_layers):
    output, currentLayer_input = [x], x

    for j in range(hidden_layers):
        activation = Sigmoid(np.dot(currentLayer_input, weights[j].T))
        output.append(activation)
        currentLayer_input = np.append(1, activation)
    
    return output

## Back Propagation:

In this function the output, weights and number of hidden layers are passed as inputs. The flow will be backwards, from output layer to the last hidden layer and follows. The weights and biases are adjusted accordingly.

In this function, error rate will be calculated. The error will be propagated backwards and the weights are adjusted. 
  - Delta Calculation : error of next layer multiplied by sigmoid derivative of current layer.
  - Weights and biases : Delta multiplied with output of previous layer, rate and sum it with weight of previous layer. Biases are updated in the same way.
  


In [5]:
def BackPropagation(y, output, weights, hidden_layers):
    final_output = output[-1]
    error = np.matrix(y - final_output) #Calculate the error at last output
    
    #Back propagate the error
    for j in range(hidden_layers, 0, -1):
        currentLayer_output = output[j]
        
        if(j > 1):
            # Add previous output
            previous_output = np.append(1, output[j-1])
        else:
            previous_output = output[0]
        
        delta = np.multiply(error, sigmoidDerivative(currentLayer_output))
        weights[j-1] += rate * np.multiply(delta.T, previous_output)

        weight = np.delete(weights[j-1], [0], axis=1) 
        error = np.dot(delta, weight) # Calculate error for current layer
    
    return weights

## Train neural network:

The right values for the weights and biases determines the strength of the predictions. The process of fine-tuning the weights and biases from the input data is known as training the Neural Network. Here, X, Y, rate and weights are passed as inputs for training the neural network.

In [6]:
def train_neuralNet(X, Y, rate, weights):
    hidden_layers = len(weights)
    for i in range(len(X)):
        x, y = X[i], Y[i]
        x = np.matrix(np.append(1, x)) # Add feature vector
        
        output = Feed_Forward(x, weights, hidden_layers)
        weights = BackPropagation(y, output, weights, hidden_layers)

    return weights

## Activation functions:

Activation function of a node defines the output of that node, given an input or set of inputs. This output is then used as input for the next node and so on until a desired solution to the original problem is found.

Activation function gets to decide which neurons will push forward the values into the next layer. Sigmoid activation function has been used for this project. It takes a value as input and outputs another value between 0 and 1. It is non-linear and easy to work with when constructing a neural network model. 

In [7]:
def Sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoidDerivative(x):
    return np.multiply(x, 1-x)

## Predict:

The input is first passed to the network. The higher value of a number indicates the most probable class. findMaxActivation() will find the maximum valued output and then corresponding index is set to 1. So, the predicted class is the final output class which we get from the neural network.

In [8]:
def predict(item, weights):
    hidden_layers = len(weights)
    item = np.append(1, item)
    
    #forward propagation
    output = Feed_Forward(item, weights, hidden_layers)
    
    final_output = output[-1].A1
    index = findMaxActivation(final_output)

    y = [0 for i in range(len(final_output))]
    y[index] = 1

    return y

In [9]:
def findMaxActivation(output):
    m, index = output[0], 0
    for i in range(1, len(output)):
        if(output[i] > m):
            m, index = output[i], i
    
    return index

## Model accuracy:

In this function, we will pass the inputs, outputs as well as the weights. If the predictionis correct, we will increase the count and find the accuracy by dividing the total correct predictions and total length of X. By doing these calculations, we can get the model accuracy.

In [10]:
def model_accuracy(X, Y, weights):
    correct_prediction = 0

    for i in range(len(X)):
        x, y = X[i], list(Y[i])
        prediction = predict(x, weights)

        if(y == prediction):
            correct_prediction += 1

    return correct_prediction / len(X)

## Note:

Now, we will use the cancer dataset and train the model using this dataset.

## Importing the Data:

In [11]:
pd.set_option('display.max_rows', 200)

df = pd.read_csv('cancer_data.csv')
df.head()

,id,clump_thickness,cell_uniformity,cellshape_uniformity,adhesion,cell_size,nuclei,chromatin,nucleoli,mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


## Preprocessing the Data:

In [12]:
#Prepare the input data
X = df[['clump_thickness', 'cell_uniformity', 'cellshape_uniformity', 'adhesion', 'cell_size', 'chromatin', 'nucleoli', 'mitoses']]
X = np.array(X)
X[:7]

array([[ 5,  1,  1,  1,  2,  3,  1,  1],
       [ 5,  4,  4,  5,  7,  3,  2,  1],
       [ 3,  1,  1,  1,  2,  3,  1,  1],
       [ 6,  8,  8,  1,  3,  3,  7,  1],
       [ 4,  1,  1,  3,  2,  3,  1,  1],
       [ 8, 10, 10,  8,  7,  9,  7,  1],
       [ 1,  1,  1,  1,  2,  3,  1,  1]])

In [13]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)

Y = df.Class
Y = one_hot_encoder.fit_transform(np.array(Y).reshape(-1, 1))
Y[:5]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])

## Splitting the data into train, validation and test data

In [14]:
train_index = int(0.75 * len(X))
X_train, X_test = X[:train_index], X[train_index:]
Y_train, Y_test = Y[:train_index], Y[train_index:]

In [15]:
val_index = int(0.65 * len(X_train))
X_train, X_val = X_train[:val_index], X_train[val_index:]
Y_train, Y_val = Y_train[:val_index], Y_train[val_index:]

## Train the neural net model

In [16]:
# Run it here
features = len(X[0]) # Number of features - using all of the features except for the sequence name
classes = len(Y[0]) # Number of classes

hidden_layers = [features, 5, 10, classes]
rate, iterations = 0.15, 500

weights = neural_network(X_train, Y_train, X_val, Y_val, iterations=iterations, nodes=hidden_layers, rate=rate)

Iteration 20
Accuracy on train data:0.9588235294117647
Accuracy on validation data:0.9565217391304348
Iteration 40
Accuracy on train data:0.9529411764705882
Accuracy on validation data:0.9293478260869565
Iteration 60
Accuracy on train data:0.961764705882353
Accuracy on validation data:0.9402173913043478
Iteration 80
Accuracy on train data:0.9323529411764706
Accuracy on validation data:0.9184782608695652
Iteration 100
Accuracy on train data:0.95
Accuracy on validation data:0.9347826086956522
Iteration 120
Accuracy on train data:0.961764705882353
Accuracy on validation data:0.9510869565217391
Iteration 140
Accuracy on train data:0.961764705882353
Accuracy on validation data:0.967391304347826
Iteration 160
Accuracy on train data:0.9529411764705882
Accuracy on validation data:0.9510869565217391
Iteration 180
Accuracy on train data:0.9205882352941176
Accuracy on validation data:0.9510869565217391
Iteration 200
Accuracy on train data:0.9558823529411765
Accuracy on validation data:0.918478260

## Accuracy on Test Data:

In [18]:
#Final testing accuracy
print("Accuracy on Test data: {}".format(model_accuracy(X_test, Y_test, weights)))

Accuracy on Test data: 0.9885714285714285


## Predictions to cross check neural network model:

---



In [19]:

print(X_test[0], list(Y_test[0]))

[3 1 1 1 2 2 1 1] [1.0, 0.0]


In [20]:
print(predict(X_test[0], weights))

[1, 0]


## Conclusion:

From these predictions, we can draw a conclusion that neural network model implemented worked pretty well on the test data and gave us a good accuracy on this dataset.